### Segmentation evaluation for training and testing set of baseline

In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, auc

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
def dice_coef(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    return (2. * np.sum(intersection)) / (np.sum(y_true) + np.sum(y_pred))

def iou_coef(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [3]:
def auc_coef(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    return auc(fpr, tpr)

def ap_coef(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return average_precision_score(y_true, y_pred)

In [4]:
def evaluate(pred_top_dir, im_top_dir, outfile, mode):
    df = pd.DataFrame()
    
    aois = sorted([f for f in os.listdir(os.path.join(im_top_dir))
                   if os.path.isdir(os.path.join(im_top_dir, f))
                  and f != 'list'])
    
    print(aois)

    auc_multi_aoi = 0.
    ap_multi_aoi = 0.
    dice_multi_aoi = 0.
    iou_multi_aoi = 0.

    for i, aoi in enumerate(aois):
        print(i, "aoi:", aoi)

        pred_dir = os.path.join(pred_top_dir, 'grouped', aoi, 'masks')
        im_dir = os.path.join(im_top_dir, aoi, 'images_masked')
        gt_dir = os.path.join(im_top_dir, aoi, 'masks')

        im_list = sorted([z for z in os.listdir(im_dir) if z.endswith('.tif')])

        auc_one_aoi = 0.
        ap_one_aoi = 0.
        dice_one_aoi = 0.
        iou_one_aoi = 0.

        for j, f in enumerate(im_list):        
            sample_mask_name = f
            if mode == 'baseline':
                sample_mask_path = os.path.join(pred_dir, sample_mask_name)
            elif mode == 'winner':
                sample_mask_path = os.path.join(pred_dir, sample_mask_name).replace('.tif', '.npy')
            sample_im_path = os.path.join(im_dir, sample_mask_name)
            sample_gt_path = os.path.join(gt_dir, sample_mask_name).replace('.tif', '_Buildings.tif')

            image = skimage.io.imread(sample_im_path)
            if mode == 'baseline':
                mask_image = skimage.io.imread(sample_mask_path)
            elif mode == 'winner':
                mask_image = np.load(sample_mask_path)
            gt_image = skimage.io.imread(sample_gt_path)
    #         print(mask_image)
    #         print("mask_image.shape:", mask_image.shape)
    #         print("min, max, mean mask image:", np.min(mask_image), np.max(mask_image), np.mean(mask_image))
    #         print(gt_image)
    #         print("gt_image.shape:", gt_image.shape)
    #         print("min, max, mean gt image:", np.min(gt_image), np.max(gt_image), np.mean(gt_image))

            norm = (mask_image - np.min(mask_image)) / (np.max(mask_image) - np.min(mask_image))

    #         print(norm)
    #         print("mask_image.shape:", norm.shape)
    #         print("min, max, mean mask image:", np.min(norm), np.max(norm), np.mean(norm))

            gt_image = gt_image / 255
            
            if mode == 'winner':
                tmp = np.zeros((1024, 1024))
                tmp[:gt_image.shape[0],:gt_image.shape[1]] = gt_image
                gt_image = np.repeat(tmp, 3, axis=0)
                gt_image = np.repeat(gt_image, 3, axis=1)
            
            auc = auc_coef(gt_image, norm)
            ap = ap_coef(gt_image, norm)
            
            auc_one_aoi += auc
            ap_one_aoi += ap
                
            norm = np.where(norm > 0.5, 1, 0)

            dice = dice_coef(gt_image, norm)
            iou = iou_coef(gt_image, norm)
            dice_one_aoi += dice
            iou_one_aoi += iou

        auc_one_aoi = auc_one_aoi / len(im_list)
        ap_one_aoi = ap_one_aoi / len(im_list)
        dice_one_aoi = dice_one_aoi / len(im_list)
        iou_one_aoi = iou_one_aoi / len(im_list)

        print("AUC: ", auc_one_aoi)
        print("AP: ", ap_one_aoi)
        print("Dice: ", dice_one_aoi)
        print("IOU: ", iou_one_aoi)
        
        row1 = {'AOI': aoi, 'AUC': auc_one_aoi, 'AP': ap_one_aoi, 'Dice': dice_one_aoi, 'IOU': iou_one_aoi}
        df = df.append(row1, ignore_index=True)
        
        auc_multi_aoi += auc_one_aoi
        ap_multi_aoi += ap_one_aoi
        dice_multi_aoi += dice_one_aoi
        iou_multi_aoi += iou_one_aoi
    
    df.to_csv(outfile, index=False, header=True)

    auc_multi_aoi = auc_multi_aoi / len(aois)
    ap_multi_aoi = ap_multi_aoi / len(aois)
    dice_multi_aoi = dice_multi_aoi / len(aois)
    iou_multi_aoi = iou_multi_aoi / len(aois)

    print("Average AUC: ", auc_multi_aoi)
    print("Average AP: ", ap_multi_aoi)
    print("Average Dice: ", dice_multi_aoi)
    print("Average IOU: ", iou_multi_aoi)

In [5]:
def group(raw_name, grouped_name):
    im_list = sorted([z for z in os.listdir(os.path.join(raw_name)) if z.endswith('.npy')])
    df = pd.DataFrame({'image': im_list})
    roots = [z.split('mosaic_')[-1].split('.npy')[0] for z in df['image'].values]
    df['root'] = roots
    # copy files
    for idx, row in df.iterrows():
        in_path_tmp = os.path.join(raw_name, row['image'])
        out_dir_tmp = os.path.join(grouped_name, row['root'], 'masks')
        os.makedirs(out_dir_tmp, exist_ok=True)
        cmd = 'cp ' + in_path_tmp + ' ' + out_dir_tmp
#         print("cmd:", cmd)
        os.system(cmd)   

In [6]:
raw_name = '/Midgard/home/hfang/temporal_CD/Berzelius_HRNet_SN7/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train/npy_compose'
grouped_name = '/Midgard/home/hfang/temporal_CD/Berzelius_HRNet_SN7/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train/grouped'
group(raw_name, grouped_name)

In [7]:
# Set prediction and image (ground truth) directories (edit appropriately)
pred_top_dir = '/Midgard/home/hfang/temporal_CD/Berzelius_HRNet_SN7/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train'
im_top_dir = '/local_storage/datasets/sn7_winner_split/test_public'

evaluate(pred_top_dir, im_top_dir, 'segmentation_tcr_v1_alpha_1.csv', mode='winner')

['L15-0387E-1276N_1549_3087_13', 'L15-0566E-1185N_2265_3451_13', 'L15-0632E-0892N_2528_4620_13', 'L15-1015E-1062N_4061_3941_13', 'L15-1200E-0847N_4802_4803_13', 'L15-1276E-1107N_5105_3761_13', 'L15-1438E-1134N_5753_3655_13', 'L15-1615E-1206N_6460_3366_13', 'L15-1690E-1211N_6763_3346_13', 'L15-1848E-0793N_7394_5018_13']
0 aoi: L15-0387E-1276N_1549_3087_13
AUC:  0.9721077359119171
AP:  0.5558607387763848
Dice:  0.34121699771229014
IOU:  0.20631386143024846
1 aoi: L15-0566E-1185N_2265_3451_13
AUC:  0.9946024812156695
AP:  0.38822660925838437
Dice:  0.2835790668776465
IOU:  0.17586796690793569
2 aoi: L15-0632E-0892N_2528_4620_13
AUC:  0.9680835473077063
AP:  0.5272666690135872
Dice:  0.423660833141783
IOU:  0.26932785881167104
3 aoi: L15-1015E-1062N_4061_3941_13
AUC:  0.9710213902354385
AP:  0.5524665228311993
Dice:  0.27431350116838
IOU:  0.15993007258618477
4 aoi: L15-1200E-0847N_4802_4803_13
AUC:  0.9098873495468927
AP:  0.5533859053101573
Dice:  0.3011546632489189
IOU:  0.1778291825633

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:951: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:951: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:951: UndefinedMetricWarning: No positive samples in y_true, true posit

AUC:  nan
AP:  nan
Dice:  0.20282994047218794
IOU:  0.12649527699356058
Average AUC:  nan
Average AP:  nan
Average Dice:  0.32545766308429686
Average IOU:  0.2023891888248388


In [8]:
def dice_iou_threshold(pred_top_dir, im_top_dir, outfile, mode):
    df = pd.DataFrame()
    avg = pd.DataFrame()
    
    aois = sorted([f for f in os.listdir(os.path.join(im_top_dir))
                   if os.path.isdir(os.path.join(im_top_dir, f))
                  and f != 'list'])

    for theta in range(11):
        threshold = theta / 10
        print(threshold)
        
        dice_multi_aoi = 0.
        iou_multi_aoi = 0.

        for i, aoi in enumerate(aois):
            print(i, "aoi:", aoi)

            pred_dir = os.path.join(pred_top_dir, 'grouped', aoi, 'masks')
            im_dir = os.path.join(im_top_dir, aoi, 'images_masked')
            gt_dir = os.path.join(im_top_dir, aoi, 'masks')

            im_list = sorted([z for z in os.listdir(im_dir) if z.endswith('.tif')])

            dice_one_aoi = 0.
            iou_one_aoi = 0.

            for j, f in enumerate(im_list):        
                sample_mask_name = f
                if mode == 'baseline':
                    sample_mask_path = os.path.join(pred_dir, sample_mask_name)
                elif mode == 'winner':
                    sample_mask_path = os.path.join(pred_dir, sample_mask_name).replace('.tif', '.npy')
                sample_im_path = os.path.join(im_dir, sample_mask_name)
                sample_gt_path = os.path.join(gt_dir, sample_mask_name).replace('.tif', '_Buildings.tif')

                image = skimage.io.imread(sample_im_path)
                if mode == 'baseline':
                    mask_image = skimage.io.imread(sample_mask_path)
                elif mode == 'winner':
                    mask_image = np.load(sample_mask_path)
                gt_image = skimage.io.imread(sample_gt_path)
        #         print(mask_image)
        #         print("mask_image.shape:", mask_image.shape)
        #         print("min, max, mean mask image:", np.min(mask_image), np.max(mask_image), np.mean(mask_image))
        #         print(gt_image)
        #         print("gt_image.shape:", gt_image.shape)
        #         print("min, max, mean gt image:", np.min(gt_image), np.max(gt_image), np.mean(gt_image))

                norm = (mask_image - np.min(mask_image)) / (np.max(mask_image) - np.min(mask_image))

        #         print(norm)
        #         print("mask_image.shape:", norm.shape)
        #         print("min, max, mean mask image:", np.min(norm), np.max(norm), np.mean(norm))

                gt_image = gt_image / 255

                if mode == 'winner':
                    tmp = np.zeros((1024, 1024))
                    tmp[:gt_image.shape[0],:gt_image.shape[1]] = gt_image
                    gt_image = np.repeat(tmp, 3, axis=0)
                    gt_image = np.repeat(gt_image, 3, axis=1)


                norm = np.where(norm > threshold, 1, 0)

                dice = dice_coef(gt_image, norm)
                iou = iou_coef(gt_image, norm)
                dice_one_aoi += dice
                iou_one_aoi += iou

            dice_one_aoi = dice_one_aoi / len(im_list)
            iou_one_aoi = iou_one_aoi / len(im_list)

            print("Dice: ", dice_one_aoi)
            print("IOU: ", iou_one_aoi)

            row1 = {'AOI': aoi, 'Dice': dice_one_aoi, 'IOU': iou_one_aoi, 'threshold': threshold}
            df = df.append(row1, ignore_index=True)

            dice_multi_aoi += dice_one_aoi
            iou_multi_aoi += iou_one_aoi

        dice_multi_aoi = dice_multi_aoi / len(aois)
        iou_multi_aoi = iou_multi_aoi / len(aois)

        print("Average Dice: ", dice_multi_aoi)
        print("Average IOU: ", iou_multi_aoi)
        
        row2 = {'Average Dice': dice_multi_aoi, 'Average IOU': iou_multi_aoi, 'threshold': threshold}
        avg = avg.append(row2, ignore_index=True)
        
    df.to_csv(outfile, index=False, header=True)
    avg.to_csv(outfile.replace('.csv', '_avg.csv'), index=False, header=True)

In [9]:
# Set prediction and image (ground truth) directories (edit appropriately)
pred_top_dir = '/Midgard/home/hfang/temporal_CD/Berzelius_HRNet_SN7/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train'
im_top_dir = '/local_storage/datasets/sn7_winner_split/test_public'

dice_iou_threshold(pred_top_dir, im_top_dir, 'segmentation_infer_threshold_tcr_v1_alpha_1.csv', mode='winner')

0.0
0 aoi: L15-0387E-1276N_1549_3087_13
Dice:  0.05912087974839937
IOU:  0.030465085916961763
1 aoi: L15-0566E-1185N_2265_3451_13
Dice:  0.008478867447901087
IOU:  0.0042724613902263
2 aoi: L15-0632E-0892N_2528_4620_13
Dice:  0.04818749091634234
IOU:  0.024688761466212954
3 aoi: L15-1015E-1062N_4061_3941_13
Dice:  0.05527082002130957
IOU:  0.028426756162299702
4 aoi: L15-1200E-0847N_4802_4803_13
Dice:  0.18087302015045392
IOU:  0.09943271736915564
5 aoi: L15-1276E-1107N_5105_3761_13
Dice:  0.04906682142129181
IOU:  0.025152781290772898
6 aoi: L15-1438E-1134N_5753_3655_13
Dice:  0.3683387224534129
IOU:  0.22688972103029237
7 aoi: L15-1615E-1206N_6460_3366_13
Dice:  0.10691707070983703
IOU:  0.05649468213620038
8 aoi: L15-1690E-1211N_6763_3346_13
Dice:  0.12348348328280251
IOU:  0.06591523392096985
9 aoi: L15-1848E-0793N_7394_5018_13
Dice:  0.01587265744289208
IOU:  0.008082581422867396
Average Dice:  0.10156098335946426
Average IOU:  0.056982078210595935
0.1
0 aoi: L15-0387E-1276N_1549_

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  


Dice:  nan
IOU:  nan
Average Dice:  nan
Average IOU:  nan
